In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import pandas as pd
import sqlalchemy
from datetime import datetime
import hashlib
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq as pdg

In [3]:
def get_gbq_data(query):
    project = 'lentochka-9bb50' 
    credentials_bq = service_account.Credentials.from_service_account_file('lentochka.json', scopes=["https://www.googleapis.com/auth/cloud-platform"],)
    pdg.context.credentials = credentials_bq
    pdg.context.project = project
    gbq_data = pdg.read_gbq(query, progress_bar_type=True)
    return gbq_data

In [4]:
# Генерация списка дат
def days_list(start_date, end_date):
    start = start_date[3:5] + '.' + start_date[0:2]  + '.' + start_date[-2:]
    end = end_date[3:5] + '.' + end_date[0:2]  + '.' + end_date[-2:]
    return [i for i in [str(pd.to_datetime(i))[:10] for i in pd.date_range(start=start, end=end, freq='d')]]

In [5]:
gbq_query = """
WITH 
rawdata AS (
    SELECT if(JSON_EXTRACT_SCALAR(payload, '$.appsflyer_id') is null, `analytics-230012`.public_functions_us.parseQueryString(payload), payload) as payload,
        'android' as platform
    FROM `hallowed-object-285011.segmentstream_202204.af_lentochka_android`
    WHERE `date` = '2021-03-01'
    
    UNION ALL 
    
    SELECT payload, 'ios' as platform
    FROM `hallowed-object-285011.segmentstream_202204.af_lentochka_ios`
    WHERE `date` = '2021-03-01' 
),

pre_table AS
(select distinct JSON_EXTRACT_SCALAR(payload, '$.appsflyer_id') AS appsflyer_id,
JSON_EXTRACT_SCALAR(payload, '$.idfa') AS idfa,
JSON_EXTRACT_SCALAR(payload, '$.idfv') AS idfv,
JSON_EXTRACT_SCALAR(payload, '$.gaid') AS gaid,
DATE(JSON_EXTRACT_SCALAR(payload, '$.event_time')) as event_date,
UNIX_SECONDS(TIMESTAMP(JSON_EXTRACT_SCALAR(payload, '$.event_time'))) AS event_time,
JSON_EXTRACT_SCALAR(payload, '$.event_name') AS event_name,
'Imported devices'
from rawdata
)

select * from pre_table
where event_name = 'install'
"""

In [6]:
list_dates = days_list('01.03.2022', '01.03.2022')

In [7]:
for el in list_dates:
    print(el)

2022-03-01


In [9]:
for el in list_dates:
    tmp_query = gbq_query.replace('{string_date}', el)
    print(get_gbq_data(tmp_query).head())

GenericGBQException: Reason: 403 Access Denied: Table hallowed-object-285011:segmentstream_202204.af_lentochka_android: User does not have permission to query table hallowed-object-285011:segmentstream_202204.af_lentochka_android.

Location: US
Job ID: 2c37ad36-7c74-4618-9fe3-510a78302d6b
